In [276]:
import sys
!{sys.executable} -m pip install scikit-learn

In [312]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import sklearn
from sklearn.model_selection import train_test_split
import math
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
import itertools



In [294]:
bt_df = pd.read_csv("blood_transfusion.csv")
bt_df.describe()

,months_since_last_donation,total_number_of_donations,total_blood_donated,months_since_first_donation,class
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [295]:
months_since_last_donation_avg = np.zeros((75, 3))

for i in range(len(months_since_last_donation_avg)):
    months_since_last_donation_avg[i, 0] = i+1

for index, row in bt_df.iterrows():
    months_since_last_donation_avg[int(row[0]), 2] += 1
    avg_class = (months_since_last_donation_avg[int(row[0]), 1] + row[4])/months_since_last_donation_avg[int(row[0]), 2]
    months_since_last_donation_avg[int(row[0]), 1] = avg_class

months_since_last_donation_fig = go.Figure()


months_since_last_donation_fig.add_trace(go.Scatter(
    x=months_since_last_donation_avg[:, 0], 
    y=months_since_last_donation_avg[:, 1],  
    mode='lines+markers',  
    name='Data Points'
))

months_since_last_donation_fig.update_layout(
    title='Chances of coming back after last donation',
    xaxis_title='Months passed since last donation',
    yaxis_title='Chances of coming back',
    xaxis=dict(range=[0, 30])
)

months_since_last_donation_fig.show()

distribution_fig = go.Figure()

for i in range(len(months_since_last_donation_avg)):
    distribution_fig.add_trace(go.Bar(
        x=months_since_last_donation_avg[:, 0],  
        y=months_since_last_donation_avg[:, 2],  
        marker = dict(color = 'red')
    ))

distribution_fig.update_layout(
    xaxis=dict(range=[0, 30])
)

distribution_fig.show()


/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/2032734748.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/2032734748.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/2032734748.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [296]:
segments_df= np.array_split(bt_df, 15)

class_distribution_fig = go.Figure()
avg_class = np.zeros((15, 2))

for i in range(len(segments_df)):
    avg_class[i][1] = sum(segments_df[i]['class'])/len(segments_df[i])
    avg_class[i][0] = i+1

class_distribution_fig = go.Figure()

class_distribution_fig.add_trace(go.Scatter(
    x=avg_class[:, 0],  
    y=avg_class[:, 1],  
    mode='lines+markers', 
    name='Data Points'
))

class_distribution_fig.update_layout(
    title='Distribution of class variable across data',
    xaxis_title='Segments of Data',
    yaxis_title='Average Value of Class',
)

class_distribution_fig.show()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.



Task 2: Data Preprocessing

In [297]:
# Data normalization

def normalize(dataColumn):
    min_value = min(dataColumn)
    max_value = max(dataColumn)
    
    normalized_data = [(x-min_value)/(max_value - min_value) for x in dataColumn]

    return normalized_data

for i in bt_df.columns:
    bt_df[i] = normalize(bt_df[i])

bt_df.describe()

,months_since_last_donation,total_number_of_donations,total_blood_donated,months_since_first_donation,class
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,0.128469,0.092137,0.092137,0.336272,0.237968
std,0.109397,0.119170,0.119170,0.253924,0.426124
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.037162,0.020408,0.020408,0.145833,0.000000
50%,0.094595,0.061224,0.061224,0.270833,0.000000
75%,0.189189,0.122449,0.122449,0.500000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


Task 3: Creating a train and test set

In [298]:
class_attribute = bt_df['class']
variables = bt_df.drop(columns = ["class"])

class_train8020, class_test8020, variables_train8020, variables_test8020= train_test_split(class_attribute, variables, test_size=0.2, train_size = 0.8, random_state=2)
class_train9010, class_test9010, variables_train9010, variables_test9010= train_test_split(class_attribute, variables, test_size=0.1, train_size = 0.9, random_state=2)

#(random_state=2, because the split produced a more equal distribution)
#potential for improvement: iterate through many random_states to find the most oprimal one

In [318]:
#Split evaluation
splits = ['Train 80-20', 'Test 80-20', 'Train 90-10', 'Test 90-10']
values = [np.mean(class_train8020), np.mean(class_test8020), np.mean(class_train9010), np.mean(class_test9010)]
groups = [0,0,1,1] 

split_evaluation_fig = go.Figure()

split_evaluation_fig.add_trace(go.Bar(
    x=np.array(splits)[np.array(groups) == 0],
    y=np.array(values)[np.array(groups) == 0],
    name='80-20 Split'                        
))

split_evaluation_fig.add_trace(go.Bar(
    x=np.array(splits)[np.array(groups) == 1],  
    y=np.array(values)[np.array(groups) == 1],      
    name='90-10 Split'                                  
))

split_evaluation_fig.update_layout(
    title='Evaluating Train-Test Splits',
    xaxis_title='Split Type',
    yaxis_title='Mean Value',
    barmode='group'
)

split_evaluation_fig.show()

Task 4: Classification Algorithms

In [299]:
#4.1: Manual implementation of a KNN classifier algorithm

def euclidean_distance(point1, point2):
    distance = 0
    for i in range(len(point1)):
        distance += (point1[i] - point2[i]) ** 2
    return math.sqrt(distance)

def get_neighbors(training_set, labels, test_instance, k):
    distances = []
    for index in range(len(training_set)):
        dist = euclidean_distance(test_instance, training_set[index])
        distances.append((labels[index], dist))
    
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return [neighbor[0] for neighbor in neighbors]

def predict_classification(training_set, labels, test_instance, k):
    neighbors = get_neighbors(training_set, labels, test_instance, k)
    count = Counter(neighbors) 
    prediction = count.most_common(1)[0][0]
    return prediction

def confusion_matrix(predicted, actual):
    predicted = np.array(predicted)
    actual = np.array(actual)

    true_positive = np.sum((predicted == 1) & (actual == 1))
    false_positive = np.sum((predicted == 1) & (actual == 0))

    true_negative = np.sum((predicted == 0) & (actual == 0))
    false_negative = np.sum((predicted == 1) & (actual == 0))

    matrix_data = {'True positive': true_positive, 'False positive': false_positive, 'True negative': true_negative, 'False negative': false_negative}
    confusion_table = pd.DataFrame(matrix_data, index=[0]) 

    return confusion_table

def precision(predicted, actual):
    
    if (int(confusion_matrix(predicted,actual)['True positive']) + int(confusion_matrix(predicted,actual)['False positive'])) == 0:
        return 0.0

    precision = int(confusion_matrix(predicted,actual)['True positive']) / int((confusion_matrix(predicted,actual)['True positive']) + int(confusion_matrix(predicted,actual)['False positive']))
    return precision

def recall(predicted, actual):
    if (int(confusion_matrix(predicted,actual)['True positive']) + int(confusion_matrix(predicted,actual)['False negative'])) == 0:
        return 0.0

    recall = int(confusion_matrix(predicted,actual)['True positive']) / (int(confusion_matrix(predicted,actual)['True positive']) + int(confusion_matrix(predicted,actual)['False negative']))
    return recall

knn8020_predicted = np.zeros(150)
knn8020_correct= np.zeros(150)

for i in range(len(knn8020_predicted)):
    predicted_label = predict_classification(variables_train8020.to_numpy(), class_train8020.to_numpy(), variables_test8020.to_numpy()[i], 20)
    knn8020_predicted[i] = predicted_label
    knn8020_correct[i] = class_test8020.to_numpy()[i]
    

evaluater_data = {'Predicted Class': knn8020_predicted, 'Correct Class': knn8020_correct}
knn8020_evaluater = pd.DataFrame(evaluater_data)

print(confusion_matrix(knn8020_evaluater['Predicted Class'], knn8020_evaluater['Correct Class']))

print(f'For a KNN Classifier with a 80-20 split, the precision is: {precision(knn8020_evaluater['Predicted Class'], knn8020_evaluater['Correct Class'])*100}% and the recall is: {recall(knn8020_evaluater['Predicted Class'], knn8020_evaluater['Correct Class'])*100}%')

knn9010_predicted = np.zeros(75)
knn9010_correct= np.zeros(75)

for i in range(len(knn9010_predicted)):
    predicted_label = predict_classification(variables_train9010.to_numpy(), class_train9010.to_numpy(), variables_test9010.to_numpy()[i], 20)
    knn9010_predicted[i] = predicted_label
    knn9010_correct[i] = class_test9010.to_numpy()[i]
    

evaluater_data = {'Predicted Class': knn9010_predicted, 'Correct Class': knn9010_correct}
knn9010_evaluater = pd.DataFrame(evaluater_data)

print(f'For a KNN Classifier with a 90-10 split, the precision is: {precision(knn9010_evaluater['Predicted Class'], knn9010_evaluater['Correct Class'])*100}% and the recall is: {recall(knn9010_evaluater['Predicted Class'], knn9010_evaluater['Correct Class'])*100}%')

   True positive  False positive  True negative  False negative
0             11               5            104               5
For a KNN Classifier with a 80-20 split, the precision is: 68.75% and the recall is: 68.75%


/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:42: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:49: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:52: FutureWarning:

Calling in

For a KNN Classifier with a 90-10 split, the precision is: 63.63636363636363% and the recall is: 63.63636363636363%


/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:42: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:49: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:52: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



In [309]:
#4.2: Naive Bayes Classifier

nb8020_correct= np.zeros(150)

nb_clf8020 = GaussianNB()
nb_clf8020.fit(variables_train8020, class_train8020)

for i in range(len(nb8020_correct)):
    nb8020_correct[i] = class_test8020.to_numpy()[i]

nb8020_predicted = nb_clf8020.predict(variables_test8020)

evaluater_data = {'Predicted Class': nb8020_predicted, 'Correct Class': nb8020_correct}
nb8020_evaluater = pd.DataFrame(evaluater_data)

print(f'For a Naive Bayes Classifier with a 80-20 split, the precision is: {int(precision(nb8020_evaluater['Predicted Class'],
    nb8020_evaluater['Correct Class'])*100)}% and the recall is: {int(recall(nb8020_evaluater['Predicted Class'], nb8020_evaluater['Correct Class'])*100)}%')

nb9010_correct= np.zeros(75)

nb_clf9010 = GaussianNB()
nb_clf9010.fit(variables_train9010, class_train9010)

for i in range(len(nb9010_correct)):
    nb9010_correct[i] = class_test9010.to_numpy()[i]

nb9010_predicted = nb_clf9010.predict(variables_test9010)

evaluater_data = {'Predicted Class': nb9010_predicted, 'Correct Class': nb9010_correct}
nb9010_evaluater = pd.DataFrame(evaluater_data)

print(f'For a Naive Bayes Classifier with a 90-10 split, the precision is: {int(precision(nb9010_evaluater['Predicted Class'],
    nb9010_evaluater['Correct Class'])*100)}% and the recall is: {int(recall(nb9010_evaluater['Predicted Class'], nb9010_evaluater['Correct Class'])*100)}%')


For a Naive Bayes Classifier with a 80-20 split, the precision is: 57% and the recall is: 57%
For a Naive Bayes Classifier with a 90-10 split, the precision is: 66% and the recall is: 66%


/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:42: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:49: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:52: FutureWarning:

Calling in

In [301]:
#4.3 Support Vector Classifier

svc8020_correct= np.zeros(150)

svc_clf8020 = SVC(gamma='auto')
svc_clf8020.fit(variables_train8020, class_train8020)

for i in range(len(svc8020_correct)):
    svc8020_correct[i] = class_test8020.to_numpy()[i]

svc8020_predicted = svc_clf8020.predict(variables_test8020)

evaluater_data = {'Predicted Class': svc8020_predicted, 'Correct Class': svc8020_correct}
svc8020_evaluater = pd.DataFrame(evaluater_data)

print(f'For a Support Vector Classifier with a 80-20 split, the precision is: {int(precision(svc8020_evaluater['Predicted Class'],
    svc8020_evaluater['Correct Class'])*100)}% and the recall is: {int(recall(svc8020_evaluater['Predicted Class'], svc8020_evaluater['Correct Class'])*100)}%')

svc9010_correct= np.zeros(75)

svc_clf9010 = SVC(gamma='auto')
svc_clf9010.fit(variables_train9010, class_train9010)

for i in range(len(svc9010_correct)):
    svc9010_correct[i] = class_test9010.to_numpy()[i]

svc9010_predicted = svc_clf9010.predict(variables_test9010)

evaluater_data = {'Predicted Class': svc9010_predicted, 'Correct Class': svc9010_correct}
svc9010_evaluater = pd.DataFrame(evaluater_data)

print(f'For a Support Vector Classifier with a 90-10 split, the precision is: {int(precision(svc9010_evaluater['Predicted Class'],
    svc9010_evaluater['Correct Class'])*100)}% and the recall is: {int(recall(svc9010_evaluater['Predicted Class'], svc9010_evaluater['Correct Class'])*100)}%')


/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:42: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:49: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



For a Support Vector Classifier with a 80-20 split, the precision is: 0% and the recall is: 0%
For a Support Vector Classifier with a 90-10 split, the precision is: 0% and the recall is: 0%


/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:42: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:49: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



In [302]:
#4.4 Multilayer Percetron (Neural Network) Classifier

mlp8020_correct= np.zeros(150)

mlp_clf8020 = MLPClassifier(max_iter=300)
mlp_clf8020.fit(variables_train8020, class_train8020)

for i in range(len(mlp8020_correct)):
    mlp8020_correct[i] = class_test8020.to_numpy()[i]

mlp8020_predicted = mlp_clf8020.predict(variables_test8020)

evaluater_data = {'Predicted Class': mlp8020_predicted, 'Correct Class': mlp8020_correct}
mlp8020_evaluater = pd.DataFrame(evaluater_data)

print(f'For a Multilayer Perceptron Classifier with a 80-20 split, the precision is: {int(precision(mlp8020_evaluater['Predicted Class'],
    mlp8020_evaluater['Correct Class'])*100)}% and the recall is: {int(recall(mlp8020_evaluater['Predicted Class'], mlp8020_evaluater['Correct Class'])*100)}%')

mlp9010_correct= np.zeros(75)

mlp_clf9010 = MLPClassifier(max_iter=300)
mlp_clf9010.fit(variables_train9010, class_train9010)

for i in range(len(mlp9010_correct)):
    mlp9010_correct[i] = class_test9010.to_numpy()[i]

mlp9010_predicted = mlp_clf9010.predict(variables_test9010)

evaluater_data = {'Predicted Class': mlp9010_predicted, 'Correct Class': mlp9010_correct}
mlp9010_evaluater = pd.DataFrame(evaluater_data)

print(f'For a Multilayer Perceptron Classifier with a 90-10 split, the precision is: {int(precision(mlp9010_evaluater['Predicted Class'],
    mlp9010_evaluater['Correct Class'])*100)}% and the recall is: {int(recall(mlp9010_evaluater['Predicted Class'], mlp9010_evaluater['Correct Class'])*100)}%')


/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:42: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:49: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:52: FutureWarning:

Calling in

For a Multilayer Perceptron Classifier with a 80-20 split, the precision is: 80% and the recall is: 80%
For a Multilayer Perceptron Classifier with a 90-10 split, the precision is: 85% and the recall is: 85%


/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:42: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:45: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:49: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/0k/1dtmj8756ps3ykn3lkd5fhqc0000gn/T/ipykernel_8118/3077510472.py:52: FutureWarning:

Calling in

Task 5: Evaluation of classification methods

In [303]:
#5.1 Manual implementation of confusion matrix

#It was implemented in 4.1, under function confusion matrix

In [304]:
#5.2 Classification Report

print("Classification Report of Naive Bayes Classifier with 80-20 split:\n", classification_report(nb8020_correct, nb8020_predicted))
print("Classification Report of Naive Bayes Classifier with 90-10 split:\n", classification_report(nb9010_correct, nb9010_predicted, zero_division = 0))
print("Classification Report of KNN Classifier with 80-20 split:\n", classification_report(knn8020_correct, knn8020_predicted))
print("Classification Report of KNN Classifier with 90-10 split:\n", classification_report(knn9010_correct, knn9010_predicted, zero_division = 0))
print("Classification Report of Support Vector Classifier with 80-20 split:\n", classification_report(svc8020_correct, svc8020_predicted))
print("Classification Report of Support Vector Classifier with 90-10 split:\n", classification_report(svc9010_correct, svc9010_predicted, zero_division = 0))
print("Classification Report of Multilayer Perceptron Classifier with 80-20 split:\n", classification_report(mlp8020_correct, mlp8020_predicted))
print("Classification Report of Multilayer Perceptron Classifier with 90-10 split:\n", classification_report(mlp9010_correct, mlp9010_predicted, zero_division = 0))

Classification Report of Naive Bayes Classifier with 80-20 split:
               precision    recall  f1-score   support

         0.0       0.74      0.97      0.84       109
         1.0       0.57      0.10      0.17        41

    accuracy                           0.73       150
   macro avg       0.66      0.54      0.50       150
weighted avg       0.69      0.73      0.66       150

Classification Report of Naive Bayes Classifier with 90-10 split:
               precision    recall  f1-score   support

         0.0       0.76      0.94      0.84        53
         1.0       0.67      0.27      0.39        22

    accuracy                           0.75        75
   macro avg       0.71      0.61      0.61        75
weighted avg       0.73      0.75      0.71        75



Classification Report of KNN Classifier with 80-20 split:
               precision    recall  f1-score   support

         0.0       0.78      0.95      0.86       109
         1.0       0.69      0.27      0.39        41

    accuracy                           0.77       150
   macro avg       0.73      0.61      0.62       150
weighted avg       0.75      0.77      0.73       150

Classification Report of KNN Classifier with 90-10 split:
               precision    recall  f1-score   support

         0.0       0.77      0.92      0.84        53
         1.0       0.64      0.32      0.42        22

    accuracy                           0.75        75
   macro avg       0.70      0.62      0.63        75
weighted avg       0.73      0.75      0.72        75

Classification Report of Support Vector Classifier with 80-20 split:
               precision    recall  f1-score   support

         0.0       0.73      1.00      0.84       109
         1.0       0.00      0.00      0.00      

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [305]:
#5.3 Fbeta Score

print("Fbeta Score of Naive Bayes Classifier with 80-20 split:", round(fbeta_score(nb8020_correct, nb8020_predicted, beta=1, average='micro'), 2))
print("Fbeta Score of Naive Bayes Classifier with 90-10 split:", round(fbeta_score(nb9010_correct, nb9010_predicted, zero_division = 0,beta=1, average='micro'), 2))
print("Fbeta Score of KNN Classifier with 80-20 split:", round(fbeta_score(knn8020_correct, knn8020_predicted,beta=1, average='micro'), 2))
print("Fbeta Score of KNN Classifier with 90-10 split:", round(fbeta_score(knn9010_correct, knn9010_predicted, zero_division = 0, beta=1, average='micro'), 2))
print("Fbeta Score of Support Vector Classifier with 80-20 split:", round(fbeta_score(svc8020_correct, svc8020_predicted, beta=1, average='micro'), 2))
print("Fbeta Score of Support Vector Classifier with 90-10 split:", round(fbeta_score(svc9010_correct, svc9010_predicted, zero_division = 0, beta=1, average='micro'), 2))
print("Fbeta Score of Multilayer Perceptron Classifier with 80-20 split:", round(fbeta_score(mlp8020_correct, mlp8020_predicted, beta=1, average='micro'), 2))
print("Fbeta Score of Multilayer Perceptron Classifier with 90-10 split:", round(fbeta_score(mlp9010_correct, mlp9010_predicted, zero_division = 0, beta=1, average='micro'), 2))

Fbeta Score of Naive Bayes Classifier with 80-20 split: 0.73
Fbeta Score of Naive Bayes Classifier with 90-10 split: 0.75
Fbeta Score of KNN Classifier with 80-20 split: 0.77
Fbeta Score of KNN Classifier with 90-10 split: 0.75
Fbeta Score of Support Vector Classifier with 80-20 split: 0.73
Fbeta Score of Support Vector Classifier with 90-10 split: 0.71
Fbeta Score of Multilayer Perceptron Classifier with 80-20 split: 0.75
Fbeta Score of Multilayer Perceptron Classifier with 90-10 split: 0.77


Task 6: Cross - Validation

In [306]:
#Finding Optimap Parameter for KNN

kf5 = KFold(n_splits=5, shuffle=True, random_state=5)

class_attribute = bt_df['class']
variables = bt_df.drop(columns = ["class"])

best_score = [0,0]

for neighbors in range(1, 25):
    mean_score = []
    for train_index, test_index in kf5.split(bt_df):
        X_train, X_test = variables.iloc[train_index], variables.iloc[test_index]
        y_train, y_test = class_attribute.iloc[train_index], class_attribute.iloc[test_index]
        
        knn_clf5 = KNeighborsClassifier(n_neighbors=neighbors)
        knn_clf5.fit(X_train, y_train)
        
        predictions = knn_clf5.predict(X_test)

        mean_score.append(fbeta_score(y_test, predictions, beta=1, average='micro'))

    if np.mean(mean_score) > best_score[0]:
        best_score[0] = np.mean(mean_score)
        best_score[1] = neighbors

print(f'The optimal nummber of neighbors in a KNN classifier for this dataset is: {best_score[1]}, with a F-Score of: {round(best_score[0], 4)}')
    


The optimal nummber of neighbors in a KNN classifier for this dataset is: 21, with a F-Score of: 0.8022


In [307]:
#Render Evaluation Scores Plot

fold_indices = []
score = []

for fold, (train_index, test_index) in enumerate(kf5.split(bt_df)):
        X_train, X_test = variables.iloc[train_index], variables.iloc[test_index]
        y_train, y_test = class_attribute.iloc[train_index], class_attribute.iloc[test_index]
        
        knn_clf5 = KNeighborsClassifier(n_neighbors=best_score[1])
        knn_clf5.fit(X_train, y_train)
        
        predictions = knn_clf5.predict(X_test)

        score.append(fbeta_score(y_test, predictions, beta=1, average='micro'))
        fold_indices.append(f'Fold {fold + 1}')

mean_score = np.mean(score)

kfold_fig = go.Figure(data=[
    go.Bar(
        x=fold_indices,
        y=score,
        name='F Score',
        marker=dict(color='blue', opacity=0.7),
    )
])

kfold_fig.add_trace(go.Scatter(
    x=fold_indices,
    y=[mean_score] * len(fold_indices),
    mode='lines',
    name= f'Mean F Score:{round(best_score[0], 4)}',
    line=dict(color='red', dash='dash')
))


kfold_fig.update_layout(
    title='K-Fold Cross-Validation F Scores',
    xaxis_title='Cross-Validation Folds',
    yaxis_title='F Score',
    yaxis=dict(range=[0, 1]) 
)

kfold_fig.show()

In [308]:

knn8020_correct= np.zeros(150)

knn_clf8020 = KNeighborsClassifier(n_neighbors=5)
knn_clf8020.fit(variables_train8020, class_train8020)

for i in range(len(knn8020_correct)):
    knn8020_correct[i] = class_test8020.to_numpy()[i]

knn8020_predicted = knn_clf8020.predict(variables_test8020)

knn9010_correct= np.zeros(75)

knn_clf9010 = KNeighborsClassifier(n_neighbors=5)
knn_clf9010.fit(variables_train9010, class_train9010)

for i in range(len(knn9010_correct)):
    knn9010_correct[i] = class_test9010.to_numpy()[i]

knn9010_predicted = knn_clf9010.predict(variables_test9010)

score_comparison = [['Optimized KFold Model', '80-20 Split', '90-10 Split'], [round(mean_score, 3), round(fbeta_score(knn8020_correct, knn8020_predicted,beta=1, average='micro'), 3), round(fbeta_score(knn9010_correct, knn9010_predicted,beta=1, average='micro'), 3)],]

kfold_fig = go.Figure(data=[
    go.Bar(
        x=score_comparison[0],
        y=score_comparison[1],
        name='F Score',
        marker=dict(color='blue', opacity=0.7),
         text=score_comparison[1]  
    )
])

kfold_fig.update_layout(
    title='Comparing KFold Model with simple KNN Models',
    xaxis_title='KNN Models',
    yaxis_title='F Score',
    yaxis=dict(range=[0, 1]) 
)

kfold_fig.show()